In [ ]:
#!unzip '/content/ind_pd_2020_1km_ASCII_XYZ.zip'

In [ ]:
import pandas as pd
import numpy as np

#Reading in population distribution data
population_dist = pd.read_csv('/content/out.csv')
population_dist.head(20)

,X,Y,Z
0,77.827916,35.503750,1.465800
1,77.836250,35.503750,1.427846
2,77.844583,35.503750,0.473976
3,77.819583,35.495417,1.207597
4,77.827916,35.495417,0.479271
5,77.836250,35.495417,0.367109
6,77.844583,35.495417,0.390246
7,78.019583,35.495417,0.000000
8,78.027916,35.495417,0.000000
9,78.036250,35.495417,0.000000


In [ ]:
len(population_dist)

4010402

In [ ]:
#Filter out rows not in the region of analysis


population_dist = population_dist[population_dist['Y'] >= 24.286006] 
population_dist = population_dist[population_dist['Y'] <= 27.521833] 
population_dist = population_dist[population_dist['X'] >= 83.320216] #Longitude
population_dist = population_dist[population_dist['X'] <= 88.293796]
population_dist = population_dist.reset_index()

#Latitude - y

population_dist.head()

,index,X,Y,Z
0,746062,84.102916,27.520417,5.963120
1,746063,84.111250,27.520417,14.401941
2,746064,84.119583,27.520417,6.269425
3,746065,84.144583,27.520417,34.482857
4,746066,88.069583,27.520417,27.543198


In [ ]:
population_dist.to_csv('out.csv')

In [ ]:
len(population_dist)

182080

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd

hospital_coords = pd.read_csv('/content/popular_hospitals_Bihar.csv')
hospital_coords['Radius'] = (hospital_coords['Popularity'] / np.max(np.array(hospital_coords['Popularity'])))*2
hospital_coords.head()

,Name,Latitude,Longitude,Popularity,Radius
0,Jeevan Jyoti Superspeciality Hospital,25.200768,85.514717,126,2.000000
1,J.P. HOSPITAL,25.197636,85.518626,83,1.317460
2,Sadar Hospital,25.205265,85.514647,73,1.158730
3,Moti Memorial Hospital Pvt. Ltd.,25.216763,85.516507,60,0.952381
4,Haddi Hospital,25.194903,85.516726,32,0.507937


In [ ]:
X = []

import math

#5 miles
# One degree of latitude equals approximately 364,000 feet (69 miles)
# I will remove everything within 20 miles of the hospital

for x in range(len(population_dist)):
  close = False
  for hospital in range(len(hospital_coords)):
    if math.sqrt((hospital_coords['Latitude'][hospital] - population_dist['Y'][x])**2 + (hospital_coords['Longitude'][hospital] - population_dist['X'][x])**2) < ((hospital_coords['Radius'][hospital]) * (1 / 69)): # 1/ 69 is for converting from miles to lat, long.
      close = True

  if close:
    continue
  for y in range(int(population_dist['Z'][x] / 10)):
    X.append([population_dist['X'][x], population_dist['Y'][x]])

X = np.array(X)

In [ ]:
Y = []

import math

# One degree of latitude equals approximately 364,000 feet (69 miles)
# I will remove everything within 20 miles of the hospital

for x in range(len(population_dist)):
  for y in range(int(population_dist['Z'][x] / 50)):
    Y.append([population_dist['X'][x], population_dist['Y'][x]])

Y = np.array(Y)

In [ ]:
len(X)

In [ ]:
len(Y)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

num_hospitals = 7

kmeans = KMeans(n_clusters=5, random_state=0).fit(X)

print(kmeans.cluster_centers_)

[[86.30483516 25.41459894]
 [83.98151727 26.5050179 ]
 [87.65159824 25.51555318]
 [84.50341015 25.06350774]
 [85.36753407 26.11049699]]


In [ ]:
kmeans.cluster_centers_

array([[86.30483516, 25.41459894],
       [83.98151727, 26.5050179 ],
       [87.65159824, 25.51555318],
       [84.50341015, 25.06350774],
       [85.36753407, 26.11049699]])

In [ ]:
#Heatmap code

import cv2
from google.colab.patches import cv2_imshow

#print(int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1)
#print(int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1)
heatmap = np.zeros((int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1, int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1))


for point in Y:
  #print(point[0])
  #heatmap[int((point[0] - np.min(np.array(population_dist['X'])))*(10**4)), int((point[1] - np.min(np.array(population_dist['Y'])))*(10**4))] += 1

  heatmap += cv2.cvtColor(cv2.rectangle(np.full((heatmap.shape[0], heatmap.shape[1], 3), 0, np.uint8), (int((point[0] - np.min(np.array(population_dist['X'])))*(10**4))-41, int((point[1] - np.min(np.array(population_dist['Y'])))*(10**4))-41), (int((point[0] - np.min(np.array(population_dist['X'])))*(10**4)) + 41, int((point[1] - np.min(np.array(population_dist['Y'])))*(10**4)) + 41), (1,1,1), -1), cv2.COLOR_BGR2GRAY)


heatmap = np.array(heatmap, dtype=np.float64)
heatmap /= np.max(heatmap)
heatmap *= 255
heatmap = cv2.blur(heatmap,(50,50))

map = cv2.imread('/content/Image 2-19-23 at 4.21 PM.jpg')

heatmap = np.array(heatmap, dtype=np.uint8)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_WINTER).astype(int)

#Accidentally generated the heatmap in reverse, so I am flipping it to account for that
heatmap = cv2.flip(heatmap, 0)

map = cv2.resize(map, (1417, 1334)).astype(int)
#print(heatmap)
heatmap = cv2.addWeighted(heatmap, 0.5, map, 0.5, 0)

cv2_imshow(heatmap)

In [ ]:
heatmap = np.array(heatmap, dtype=np.uint8)
cv2_imshow(cv2.resize(heatmap, (1417//2, 1334//2)))

In [ ]:
final_heatmap = heatmap
final_heatmap = np.array(final_heatmap, dtype=np.float64)

#empty = np.zeros((int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1, int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1, 3))


for hospital in range(len(hospital_coords)):
  empty = np.zeros((int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1, int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1, 3))
  circle = cv2.circle(empty, (int((hospital_coords['Longitude'][hospital] - np.min(np.array(population_dist['X'])))*(10**4)), int((hospital_coords['Latitude'][hospital] - np.min(np.array(population_dist['Y'])))*(10**4))), (int)(140*hospital_coords['Radius'][hospital]), (0,150,255), 2)
  #print(circle.shape)
  circle = cv2.resize(circle, (1417, 1334))
  #print(final_heatmap.shape)
  #print(circle.shape)
  circle = cv2.flip(circle, 0)
  final_heatmap += circle

  empty = np.zeros((int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1, int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1, 3))
  center_circle = cv2.circle(empty, (int((hospital_coords['Longitude'][hospital] - np.min(np.array(population_dist['X'])))*(10**4)), int((hospital_coords['Latitude'][hospital] - np.min(np.array(population_dist['Y'])))*(10**4))), 10, (0,150,255), -1)
  #print(circle.shape)
  center_circle = cv2.resize(center_circle, (1417, 1334))
  #print(final_heatmap.shape)
  #print(circle.shape)
  center_circle = cv2.flip(center_circle, 0)
  final_heatmap += center_circle



empty = np.zeros((int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1, int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1, 3))


for pred_hospital in kmeans.cluster_centers_:
  empty = np.zeros((int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1, int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1, 3))
  circle = cv2.circle(empty, (int((pred_hospital[0] - np.min(np.array(population_dist['X'])))*(10**4)), int((pred_hospital[1] - np.min(np.array(population_dist['Y'])))*(10**4))), 150, (0,0,255), 2)
  #print(circle.shape)
  circle = cv2.resize(circle, (1417, 1334))
  #print(final_heatmap.shape)
  #print(circle.shape)
  circle = cv2.flip(circle, 0)

  #cv2_imshow(circle)

  final_heatmap += circle

  empty = np.zeros((int((np.max(np.array(population_dist['X'])) - np.min(np.array(population_dist['X'])))*(10**4)) + 1, int((np.max(np.array(population_dist['Y'])) - np.min(np.array(population_dist['Y'])))*(10**4))+1, 3))
  center_circle = cv2.circle(empty, (int((pred_hospital[0] - np.min(np.array(population_dist['X'])))*(10**4)), int((pred_hospital[1] - np.min(np.array(population_dist['Y'])))*(10**4))), 10, (0,0,255), -1)
  #print(circle.shape)
  center_circle = cv2.resize(center_circle, (1417, 1334))
  #print(final_heatmap.shape)
  #print(circle.shape)
  center_circle = cv2.flip(center_circle, 0)

  final_heatmap += center_circle

cv2_imshow(final_heatmap)

In [ ]:
import folium


m = folium.Map(location=[25.4484, 78.5685], zoom_start=12)

for hospital in kmeans.cluster_centers_: #kmeans.cluster_centers_
    print([hospital[1], hospital[0]])
    marker = folium.Marker(location=[hospital[1], hospital[0]], popup='Proposed Hospital', icon=folium.Icon(color='red', icon_color='white', icon='glyphicon glyphicon-heart-empty')) #popup=company['label']
    marker.add_to(m)
    folium.Circle(location=[hospital[1], hospital[0]], fill_color='red', radius=1570, weight=2, color="red").add_to(m) #Multiplied by 1000 for scaling


for x in range(len(hospital_coords)): #kmeans.cluster_centers_
    marker = folium.Marker(location=[hospital_coords['Latitude'][x], hospital_coords['Longitude'][x]], popup=hospital_coords['Name'][x], icon=folium.Icon(color='blue', icon_color='white', icon='glyphicon glyphicon-heart-empty'))
    marker.add_to(m)
    folium.Circle(location=[hospital_coords['Latitude'][x], hospital_coords['Longitude'][x]], fill_color='blue', radius=hospital_coords['Radius'][x]*1500, weight=2, color="blue").add_to(m)


folium.Rectangle([(25.363029,78.489075), (25.519207,78.635502)], fill_color="green", color="green", weight=1).add_to(m)

m

In [ ]:
m.save("index_Bihar.html")